In [ ]:
import pandas as pd
from binance.client import Client
from datetime import datetime
from key import cliente

In [ ]:
import pandas as pd
from datetime import datetime
from key import cliente  # Importa o cliente direto da key
from binance.exceptions import BinanceAPIException  # Importa a exceção para tratar símbolos inválidos

# Função para converter timestamps para datas legíveis
def to_date(timestamp):
    return datetime.fromtimestamp(timestamp / 1000).strftime('%Y-%m-%d %H:%M:%S')

# 1. Função para obter o histórico de depósitos para todos os ativos com saldo > 0
def get_deposits():
    deposits = cliente.get_deposit_history()
    if not deposits:
        return pd.DataFrame(columns=['asset', 'amount']), pd.DataFrame(columns=['asset', 'amount'])

    df_deposits = pd.DataFrame(deposits)
    df_deposits['amount'] = df_deposits['amount'].astype(float)
    df_deposits['insertTime'] = pd.to_datetime(df_deposits['insertTime'], unit='ms')
    
    # Agrupando os depósitos por ativo
    total_deposits = df_deposits.groupby('asset')['amount'].sum().reset_index()
    return df_deposits, total_deposits

# 2. Função para obter o histórico de retiradas para todos os ativos com saldo > 0
def get_withdrawals():
    withdrawals = cliente.get_withdraw_history()
    if not withdrawals:
        return pd.DataFrame(columns=['asset', 'amount']), pd.DataFrame(columns=['asset', 'amount'])

    df_withdrawals = pd.DataFrame(withdrawals)
    df_withdrawals['amount'] = df_withdrawals['amount'].astype(float)
    df_withdrawals['applyTime'] = pd.to_datetime(df_withdrawals['applyTime'], unit='ms')
    
    # Agrupando as retiradas por ativo
    total_withdrawals = df_withdrawals.groupby('asset')['amount'].sum().reset_index()
    return df_withdrawals, total_withdrawals

# 3. Função para obter o histórico de trades e calcular lucro/prejuízo para cada ativo com saldo > 0
def get_trades():
    all_trades = []
    assets = cliente.get_account()['balances']
    
    # Filtrar apenas os ativos com saldo > 0
    assets_with_balance = [asset['asset'] for asset in assets if float(asset['free']) + float(asset['locked']) > 0]
    
    for asset in assets_with_balance:
        symbol = f"{asset}USDT"
        
        # Verificar se o símbolo existe antes de buscar as ordens
        try:
            trades = cliente.get_all_orders(symbol=symbol)
        except BinanceAPIException as e:
            if e.code == -1121:  # Código de erro para símbolo inválido
                print(f"Par de trading inválido para o ativo {asset}, ignorando...")
                continue
            else:
                raise e  # Repassar outros erros
            
        df_trades = pd.DataFrame(trades)
        
        if df_trades.empty:
            continue
        
        df_trades = df_trades[df_trades['status'] == 'FILLED']
        df_trades['price'] = df_trades['price'].astype(float)
        df_trades['origQty'] = df_trades['origQty'].astype(float)
        df_trades['quoteQty'] = df_trades['cummulativeQuoteQty'].astype(float)
        df_trades['time'] = pd.to_datetime(df_trades['time'], unit='ms')

        # Calcula lucro/prejuízo
        df_trades['side'] = df_trades['side'].apply(lambda x: 1 if x == 'BUY' else -1)
        df_trades['value'] = df_trades['price'] * df_trades['origQty'] * df_trades['side']
        profit_loss = df_trades['value'].sum()
        
        all_trades.append({'asset': asset, 'profit_loss': profit_loss})
    
    df_all_trades = pd.DataFrame(all_trades)
    return df_all_trades

# 4. Função para obter o saldo atual de cada ativo com saldo > 0
def get_balance():
    account_info = cliente.get_account()
    balances = account_info['balances']
    
    df_balances = pd.DataFrame(balances)
    df_balances['free'] = df_balances['free'].astype(float)
    df_balances['locked'] = df_balances['locked'].astype(float)
    df_balances['total'] = df_balances['free'] + df_balances['locked']
    
    # Filtra apenas os ativos com saldo > 0
    df_balances = df_balances[df_balances['total'] > 0][['asset', 'total']]
    return df_balances

# Coletando todos os dados
df_deposits, total_deposits = get_deposits()
df_withdrawals, total_withdrawals = get_withdrawals()
df_trades = get_trades()
df_balances = get_balance()

# Mesclando todos os dados em um resumo final
df_summary = df_balances.merge(total_deposits, on='asset', how='left').merge(
    total_withdrawals, on='asset', how='left').merge(
    df_trades, on='asset', how='left')

# Renomeando e preenchendo valores ausentes com 0
df_summary.columns = ['Asset', 'Current Balance', 'Total Deposits', 'Total Withdrawals', 'Profit/Loss from Trades']
df_summary = df_summary.fillna(0)

# Calculando saldo final por ativo e adicionando colunas extras
df_summary['Net Profit'] = df_summary['Profit/Loss from Trades'] - df_summary['Total Withdrawals'] + df_summary['Total Deposits']
df_summary['Net Balance (Balance - Deposits)'] = df_summary['Current Balance'] - df_summary['Total Deposits']

# Exibindo o resumo final
print("Resumo Final por Ativo:")
print(df_summary)


Par de trading inválido para o ativo USDT, ignorando...


TypeError: Can only merge Series or DataFrame objects, a <class 'int'> was passed

In [9]:
df_balances

,asset,free,locked,total
0,BTC,0.000024,0.0,0.000024
1,LTC,0.000000,0.0,0.000000
2,ETH,0.000076,0.0,0.000076
3,NEO,0.000000,0.0,0.000000
4,BNB,0.033354,0.0,0.033354
...,...,...,...,...
612,1MBABYDOGE,0.000000,0.0,0.000000
613,EIGEN,0.000000,0.0,0.000000
614,SCR,0.000000,0.0,0.000000
615,KAIA,0.000000,0.0,0.000000
